In [1]:
import pandas as pd
import numpy as np
from PyQt6.QtCharts import QChart, QBarSeries


In [2]:
pd.set_option('display.max_columns', None)
# 전체민원 시트 불러오기
df = pd.read_excel("/Users/choenamho/Downloads/test.xlsx", sheet_name="전체민원")     
df.head()         # 상위 5개 행 확인
df.columns        # 컬럼명 확인
     # 결측치/데이터 타입 확인

Index(['접수처', '접수일', '처리기한', '처리완료일', '민원유형', 'PG사', '카드사', '승인일', '승인번호',
       '승인금액', '할부', '가맹점\n사업자번호', '가맹점명', 'MID', 'TID명', 'TID\n사업자번호', '요청사항',
       '민원인', '민원인 연락처', '민원내용', '처리내용', '처리결과', '취소여부', '취소일자', '입금금액',
       '입금일자', '입금여부', '요청사항.1', '민원인.1', '처리상태'],
      dtype='object')

In [11]:
df["접수년"] = df["접수일"].dt.year
df["월"] = df["접수일"].dt.month
df["일"] = df["접수일"].dt.day
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 33 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   접수처        387 non-null    object        
 1   접수일        387 non-null    datetime64[ns]
 2   처리기한       387 non-null    datetime64[ns]
 3   처리완료일      337 non-null    object        
 4   민원유형       387 non-null    object        
 5   PG사        387 non-null    object        
 6   카드사        387 non-null    object        
 7   승인일        387 non-null    object        
 8   승인번호       387 non-null    int64         
 9   승인금액       387 non-null    int64         
 10  할부         387 non-null    int64         
 11  가맹점
사업자번호  387 non-null    object        
 12  가맹점명       387 non-null    object        
 13  MID        387 non-null    object        
 14  TID명       387 non-null    object        
 15  TID
사업자번호  386 non-null    object        
 16  요청사항       51 non-null     object        
 1

In [8]:
# 연도 콤보박스 리스트
years = sorted(df["접수년"].unique())
print("✅ 연도 콤보:", years)

# 최신 연도 기준 월 콤보
latest_year = years[-1]
months = sorted(df[df["접수년"] == latest_year]["월"].unique())
print(f"✅ 월 콤보 ({latest_year}):", months)

# 최신 연/월 기준 일 콤보
latest_month = months[-1]
days = sorted(df[(df["접수년"] == latest_year) & (df["월"] == latest_month)]["일"].unique())
print(f"✅ 일 콤보 ({latest_year}-{latest_month}):", days)

# 기본값
print(f"✅ 기본 선택값: {latest_year}-{latest_month}-{days[-1]}")

# 예: 선택된 날짜 기준 필터
filtered_df = df[
    (df["접수년"] == latest_year) &
    (df["월"] == latest_month) &
    (df["일"] == days[-1])
]
filtered_df.head()

# 필요한 집계 작업
grouped = df.groupby(["접수일", "가맹점명", "TID명"]).agg(
    민원건수=("TID명", "count"),
    기한내처리건수=("처리상태", lambda x: x.str.lower().eq("y").sum())
).reset_index()

# 회신율 계산
grouped["회신율"] = ((grouped["기한내처리건수"] / grouped["민원건수"]) * 100).round(1).astype(str) + "%"

# 날짜별 민원비중 계산
total_by_date = grouped.groupby("접수일")["민원건수"].transform("sum")
grouped["날짜별민원비중"] = ((grouped["민원건수"] / total_by_date) * 100).round(1).astype(str) + "%"

# 컬럼 순서 정렬
column_order = ["접수일", "가맹점명", "TID명", "날짜별민원비중", "민원건수", "기한내처리건수", "회신율"]
grouped = grouped[column_order]

# 결과 미리보기
grouped.head()

✅ 연도 콤보: [2025]
✅ 월 콤보 (2025): [6, 7, 8]
✅ 일 콤보 (2025-8): [1]
✅ 기본 선택값: 2025-8-1


,접수일,가맹점명,TID명,날짜별민원비중,민원건수,기한내처리건수,회신율
0,2025-06-25,피오나,(주)메가플러스,6.2%,1,1,100.0%
1,2025-06-25,피오나,스마트해운항공 주식회사,37.5%,6,6,100.0%
2,2025-06-25,피오나,주식회사 리치트레이드,12.5%,2,2,100.0%
3,2025-06-25,피오나,주식회사 서원트레이딩,6.2%,1,1,100.0%
4,2025-06-25,피오나,주식회사 스마트빌드,37.5%,6,6,100.0%
